In [1]:
with open('api.txt','r') as r:
    TEXT = r.read()

In [2]:
from ecommerce.src_base_scraper.scraper_builder import ScraperManagerBuilder
from ecommerce.src_base_scraper.requests_handler import RequestsHandler
from ecommerce.scraper_data.functions import get_pages,get_product,get_product_data

In [6]:
from ecommerce.data_parsing.iterator_parser import EcommerceFuncGenerator

In [20]:
handler = RequestsHandler.create_instance(None)
scrfunc = EcommerceFuncGenerator(requests_handler=handler,
                                 first_url = 'https://www.dedeman.ro/ro/cazi-de-baie/c/69?page={2}',
                                 max_page = 18
                                 )
func = scrfunc.generate_url_function()
builder = ScraperManagerBuilder(requests_handler = handler,
                      initial_url='https://www.dedeman.ro/ro/cazi-de-baie/c/69',
                      initial_node_function = func,
                      node_functions=[get_product],
                      product_functions = get_product_data
                      )

In [21]:
scraper = builder.build()

In [22]:
a = scraper.get_data(parallel_processing_flag=True,depth_parallelism=0)

In [23]:
for i in a:
    print(i)

https://www.dedeman.ro/ro/birou-cu-sertare-si-dulap-liber-pentru-calculator-alb-mat-135-x-50-x-75-5-cm-1c/p/8051485
https://www.dedeman.ro/ro/birou-cu-sertare-sardinia-rs-pentru-calculator-stejar-artisan-137-5-x-56-x-75-cm-1c/p/8066873
https://www.dedeman.ro/ro/scaun-birou-copii-fix-lemn-stratificat-fag/p/8006059
https://www.dedeman.ro/ro/birou-cu-dulap-rey-pentru-calculator-alb-mat-100-x-50-x-74-5-cm-1c/p/8054383
https://www.dedeman.ro/ro/etajera-tomar-3-ob-hdf-alba-70-x-24-5-x-107-5-cm-1c/p/8093487
https://www.dedeman.ro/ro/bec-led-hoff-clasic-e27-9w-450lm-lumina-rece-6000-k-cu-acumulator/p/1047678
https://www.dedeman.ro/ro/etajera-perete-valencia-zp-mdf-stejar-artisan-gri-grafit-folie-mata-gri-146-x-19-5-x-24-cm-1c/p/8093414
https://www.dedeman.ro/ro/scaun-birou-directorial-leganza-rotativ-stofa-c14-albastru-inchis/p/8012793
https://www.dedeman.ro/ro/veioza-de-sare-rocky-4120-1-x-e14-15w-portocalie-moderna/p/1037485
https://www.dedeman.ro/ro/birou-calculator-cu-sertar-si-dulap-stefa